In [1]:
import gurobipy as gp
from gurobipy import *

In [26]:
def getTableau(model, ibfs):
    constrs = model.getConstrs()
    vars = model.getVars()
    m = model.NumConstrs
    n = model.NumVars
    tableau = [[0 for j in range(n+2)] for i in range(m+1)] #z and rhs column makes n+2, objective row makes m+1
    tableau[0][0] = 1
    #objective row
    for i in range(n):
        tableau[0][i+1] = -1*vars[i].obj
    #constraint rows
    for i in range(m):
        for j in range(n):
            c = constrs[i]
            v = vars[i]
            tableau[i+1][j+1] = model.getCoeff(c, v)
        tableau[i+1][-1] = constrs[i].rhs
    #update based on ibfs
    rowlabel = ['' for i in range(m+1)]
    rowlabel[0] = 'z'
    for j in range(n):
        if ibfs[j] == 0:
            continue
        for i in range(1, m+1):
            print(tableau)
            if tableau[i][j+1] != 0 and len(rowlabel[i]) == 0:
                rowlabel[i] = vars[j].VarName
                if tableau[i][j+1] != 1:
                    mult = 1/tableau[i][j+1]
                    for j2 in range(n+2):
                        tableau[i][j2] *= mult
                for i2 in range(1, m+1):
                    if i2 == i or tableau[i2][j+1] == 0:
                        continue
                    mult = tableau[i2][j+1]
                    for j2 in range(n+2):
                        tableau[i2][j2] -= mult*tableau[i][j2]
    while True:
        least = 0
        leasto = -1
        for j in range(1, n+1):
            if tableau[0][j] < least:
                least = tableau[0][j]
                leasto = j
        if leasto < 0:
            print('optimal value is', str(tableau[0][-1]))
            return (tableau, rowlabel)
        lowrat = 1e20
        lowown = -1
        for i in range(1, m+1):
            if tableau[i][leasto]*tableau[i][-1] <= 0:
                continue
            if tableau[i][-1]/tableau[i][leasto] < lowrat:
                lowrat = tableau[i][-1]/tableau[i][leasto]
                lowown = i
        rowlabel[lowown] = vars[leasto-1].VarName
        if tableau[lowown][leasto] != 1:
            rat = tableau[lowown][leasto]
            for j in range(n+2):
                tableau[lowown][j] = tableau[lowown][j]/rat
        for i in range(m+1):
            if i == lowown or tableau[i][leasto] == 0:
                continue
            mult = tableau[i][leasto]
            for j in range(n+2):
                tableau[i][j] -= mult*tableau[lowown][j]
            
    
    
    
    


In [27]:
#assumes model is maximization in Ax <= b form, x >= 0, b >= 0
fname = input('Please input file name correctly:')
m = gp.read(fname)
constrs = m.getConstrs()
slack_var = 0
ibfs = [0 for i in range(m.numVars)]
for c in constrs:
    slack_var += 1
    s = m.addVar(name= 's'+str(slack_var))
    ibfs.append(c.rhs)
    m.addConstr(quicksum(m.getCoeff(c, var)*var for var in m.getVars())+s == c.rhs)
    m.remove(c)
m.update()
m.optimize()
flag = True
tol = 1e-10

cuts = 0

(tableau, rowlabels) = getTableau(m, ibfs)
    
    



Read MPS format model from file test_model_4.mps
Reading time = 0.00 seconds
Test: 7 rows, 5 columns, 14 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 21 nonzeros
Model fingerprint: 0x57605beb
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 6 rows and 9 columns
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e+01   2.200000e+00   0.000000e+00      0s
       1    3.2875000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.287500000e+01
[[1, -7.0, -2.0, -4.0, -10.0, -10.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0], [0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [19]:
print(tableau)

[[1.0, 3.0, 8.0, 6.0, 0.0, 0.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], [0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0], [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 46.0]]


In [20]:
print(rowlabels)

['z', 'x[3]', '', '', '', '', '', '']
